#### 
### <font color="GREEN">Introduction to Web Scraping</font>
# <font color="RED">Amazon Reviews Scraping</font>

In [85]:
# Import packages
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

In [86]:
# Header to set the requests as a browser requests
headers = {
    'authority': 'www.amazon.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'en-US,en;q=0.9,bn;q=0.8',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Google Chrome";v="102"',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}

In [87]:
# URL of The amazon Review page
# reviews_url = 'https://www.amazon.com/Legendary-Whitetails-Journeyman-Jacket-Tarmac/product-reviews/B013KW38RQ/'
reviews_url = 'https://www.amazon.in/Sensodyne-Sensitive-Toothbrush-Pack/product-reviews/B018FPE7QE/'

In [88]:
# Define Page No
len_page = 10

### <font color="red">Functions</font>

In [89]:
# Extra Data as Html object from amazon Review page
def reviewsHtml(url, len_page):
    
    # Empty List define to store all pages html data
    soups = []
    
    # Loop for gather all 3000 reviews from 300 pages via range
    for page_no in range(1, len_page + 1):
        
        # parameter set as page no to the requests body
        params = {
            'ie': 'UTF8',
            'reviewerType': 'all_reviews',
            'filterByStar': 'critical',
            'pageNumber': page_no,
        }
        
        # Request make for each page
        response = requests.get(url, headers=headers, params=params)
        
        # Save Html object by using BeautifulSoup4 and lxml parser
        soup = BeautifulSoup(response.text, 'lxml')
        
        # Add single Html page data in master soups list
        soups.append(soup)
        
    return soups

In [90]:
# Grab Reviews name, description, date, stars, title from HTML
def getReviews(html_data):

    # Create Empty list to Hold all data
    data_dicts = []
    
    # Select all Reviews BOX html using css selector
    boxes = html_data.select('div[data-hook="review"]')
    
    # Iterate all Reviews BOX 
    for box in boxes:
        
        # Select Name using css selector and cleaning text using strip()
        # If Value is empty define value with 'N/A' for all.
        try:
            name = box.select_one('[class="a-profile-name"]').text.strip()
        except Exception as e:
            name = 'N/A'

        try:
            stars = box.select_one('[data-hook="review-star-rating"]').text.strip().split(' out')[0]
        except Exception as e:
            stars = 'N/A'   

        try:
            title = box.select_one('[data-hook="review-title"]').text.strip()
        except Exception as e:
            title = 'N/A'

        try:
            # Convert date str to dd/mm/yyy format
            datetime_str = box.select_one('[data-hook="review-date"]').text.strip().split(' on ')[-1]
            date = datetime.strptime(datetime_str, '%B %d, %Y').strftime("%d/%m/%Y")
        except Exception as e:
            date = 'N/A'

        try:
            description = box.select_one('[data-hook="review-body"]').text.strip()
        except Exception as e:
            description = 'N/A'

        # create Dictionary with al review data 
        data_dict = {
            'Name' : name,
            'Stars' : stars,
            'Title' : title,
            'Date' : date,
            'Description' : description
        }

        # Add Dictionary in master empty List
        data_dicts.append(data_dict)
    
    return data_dicts

### <font color="red">Data Process</font>

In [91]:
# Grab all HTML
html_datas = reviewsHtml(reviews_url, len_page)

In [92]:
# Empty List to Hold all reviews data
reviews = []

In [93]:
# Iterate all Html page 
for html_data in html_datas:
    
    # Grab review data
    review = getReviews(html_data)
    
    # add review data in reviews empty list
    reviews += review

In [94]:
# Create a dataframe with reviews Data
df_reviews = pd.DataFrame(reviews)

In [95]:
df_reviews

,Name,Stars,Title,Date,Description
0,Kindle Customer,3.0,3.0 out of 5 stars\nBrush,N/A,Ok
1,Mr 12,2.0,2.0 out of 5 stars\nPoor delivery,N/A,"The product is good but delivery is not, one ..."
2,Hema Shankar,3.0,3.0 out of 5 stars\nTooth Brush,N/A,Good reach because of the taper. Not so good c...
3,ANKIT SINGH,1.0,1.0 out of 5 stars\nDuplicate product,N/A,Don't Buy it . Duplicate product is selling a...
4,"P. Ganesan, Coimbatore",3.0,3.0 out of 5 stars\nNot aspersions described.,N/A,It was stated as with soft bristles. Actually ...
...,...,...,...,...,...
95,Rukshar khan,1.0,1.0 out of 5 stars\nBad,N/A,Bad
96,SV,1.0,1.0 out of 5 stars\nNot very good,N/A,"Bought for my mother, was not very helpful as ..."
97,Jharna Malviya Sharma,1.0,1.0 out of 5 stars\nWorst quality,N/A,It is not soft like market product. Don't ord...
98,Mohan,1.0,1.0 out of 5 stars\nWrong price,N/A,It was mentioned as 100 rupees worth toothbrus...


In [96]:
# Save data
df_reviews.to_csv('reviews.csv', index=False)